Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
from pyspark.sql.functions import udf,count,to_date,datediff,size, split, array_distinct
from pyspark.sql.types import StringType,IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col

In [3]:
from pyspark.sql import SparkSession

# Create SparkSession object
spark = SparkSession.builder.appName("FlightDataAnalysis").getOrCreate()

# Read flight data from CSV file from a google bucket
df = spark.read.csv("gs://flight-data-bucket-551/itineraries_random_2M.csv", header=True)

23/12/05 21:33:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# Convertingit into Time Strap
def convert_duration(duration):
    hours_match = re.search(r'(\d+)H', duration)
    minutes_match = re.search(r'(\d+)M', duration)
    
    if hours_match and minutes_match:
        hours = int(hours_match.group(1))
        minutes = int(minutes_match.group(1))
        return datetime.strptime(f"{hours}:{minutes}", '%H:%M').time()
    else:
        return None  # Handle cases where the pattern is not found
convert_duration_udf = udf(convert_duration, StringType())
df = df.withColumn("travelDuration", convert_duration_udf("travelDuration"))


In [10]:
df.select('travelDuration').show(truncate=False)

+-----------------------------------------------------+
|travelDuration                                       |
+-----------------------------------------------------+
|Time: 4 hours, 19 minutes, 0 seconds, 0 microseconds |
|Time: 8 hours, 10 minutes, 0 seconds, 0 microseconds |
|Time: 11 hours, 54 minutes, 0 seconds, 0 microseconds|
|Time: 1 hours, 20 minutes, 0 seconds, 0 microseconds |
|Time: 6 hours, 41 minutes, 0 seconds, 0 microseconds |
|Time: 2 hours, 6 minutes, 0 seconds, 0 microseconds  |
|Time: 7 hours, 28 minutes, 0 seconds, 0 microseconds |
|Time: 3 hours, 30 minutes, 0 seconds, 0 microseconds |
|Time: 3 hours, 20 minutes, 0 seconds, 0 microseconds |
|Time: 1 hours, 51 minutes, 0 seconds, 0 microseconds |
|Time: 11 hours, 26 minutes, 0 seconds, 0 microseconds|
|Time: 5 hours, 8 minutes, 0 seconds, 0 microseconds  |
|Time: 6 hours, 20 minutes, 0 seconds, 0 microseconds |
|Time: 5 hours, 55 minutes, 0 seconds, 0 microseconds |
|Time: 9 hours, 47 minutes, 0 seconds, 0 microse

In [5]:
# Applying label Encoding
columns_to_encode = ["startingAirport", "destinationAirport"]
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_encoded").fit(df) for col in columns_to_encode]
for indexer in indexers:
    df = indexer.transform(df)

In [ ]:
df.withColumn('startingAirport_encoded',df_encoded.select('startingAirport_encoded'))
df.withColumn('destinationAirport_encoded',df_encoded.select('destinationAirport_encoded'))

In [32]:
 
 df.groupBy("segmentsAirlineName").agg(count("*").alias("count")).orderBy("count", ascending=False).show(truncate=False)

+-------------------------------------------------------+------+
|segmentsAirlineName                                    |count |
+-------------------------------------------------------+------+
|American Airlines||American Airlines                   |351795|
|Delta||Delta                                           |282972|
|United||United                                         |261716|
|American Airlines                                      |199675|
|Delta                                                  |129375|
|United                                                 |124487|
|Spirit Airlines||Spirit Airlines                       |109533|
|JetBlue Airways                                        |92784 |
|JetBlue Airways||JetBlue Airways                       |68418 |
|Delta||United                                          |37229 |
|Alaska Airlines||Alaska Airlines                       |37066 |
|United||Delta                                          |36857 |
|United||United||Delta   

In [38]:
df.select('flightDate').show()

+----------+
|flightDate|
+----------+
|2022-06-18|
|2022-06-24|
|2022-06-30|
|2022-08-04|
|2022-07-20|
|2022-04-30|
|2022-06-17|
|2022-07-03|
|2022-05-01|
|2022-05-28|
|2022-07-05|
|2022-07-12|
|2022-06-28|
|2022-07-13|
|2022-06-10|
|2022-06-13|
|2022-06-02|
|2022-05-03|
|2022-07-08|
|2022-04-30|
+----------+
only showing top 20 rows



In [7]:
df = df.withColumn("searchDate", to_date("searchDate", "yyyy-MM-dd"))
df = df.withColumn("flightDate", to_date("flightDate", "yyyy-MM-dd"))

In [8]:
df = df.withColumn("searchDate", df["searchDate"].cast("date"))
df = df.withColumn("flightDate", df["flightDate"].cast("date"))
# Calculate time gap in days
df = df.withColumn("time_gap", datediff("flightDate", "searchDate").cast(IntegerType()))

In [41]:
df.select("time_gap").show(truncate=False)

+--------+
|time_gap|
+--------+
|20      |
|52      |
|59      |
|55      |
|26      |
|11      |
|51      |
|14      |
|3       |
|40      |
|28      |
|55      |
|17      |
|36      |
|13      |
|57      |
|44      |
|12      |
|56      |
|9       |
+--------+
only showing top 20 rows



In [9]:
fdf_nonstop = df.filter(df['isNonStop'] == True)

In [43]:
 fdf_nonstop.groupBy("segmentsAirlineName").agg(count("*").alias("count")).orderBy("count", ascending=False).show(truncate=False)

+-------------------+------+
|segmentsAirlineName|count |
+-------------------+------+
|American Airlines  |199675|
|Delta              |129375|
|United             |124487|
|JetBlue Airways    |92784 |
|Spirit Airlines    |18372 |
|Alaska Airlines    |12481 |
|Frontier Airlines  |11850 |
+-------------------+------+



In [10]:
fdf_stops = df.filter(df['isNonStop'] == False)
fdf_stops.groupBy("segmentsAirlineName").agg(count("*").alias("count")).orderBy("count", ascending=False).show(truncate=False)

+-------------------------------------------------------+------+
|segmentsAirlineName                                    |count |
+-------------------------------------------------------+------+
|American Airlines||American Airlines                   |351795|
|Delta||Delta                                           |282972|
|United||United                                         |261716|
|Spirit Airlines||Spirit Airlines                       |109533|
|JetBlue Airways||JetBlue Airways                       |68418 |
|Delta||United                                          |37229 |
|Alaska Airlines||Alaska Airlines                       |37066 |
|United||Delta                                          |36857 |
|United||United||Delta                                  |22831 |
|American Airlines||American Airlines||American Airlines|22761 |
|Delta||Delta||Delta                                    |21274 |
|Delta||United||United                                  |19493 |
|Frontier Airlines||Front

In [11]:
fdf_stops_same_airline = df.filter(size(array_distinct(split(col("segmentsAirlineName"), "\|\|"))) == 1)

In [48]:
fdf_stops_same_airline.groupBy("segmentsAirlineName").agg(count("*").alias("count")).orderBy("count", ascending=False).show(truncate=False)

+-------------------------------------------------------+------+
|segmentsAirlineName                                    |count |
+-------------------------------------------------------+------+
|American Airlines||American Airlines                   |351795|
|Delta||Delta                                           |282972|
|United||United                                         |261716|
|American Airlines                                      |199675|
|Delta                                                  |129375|
|United                                                 |124487|
|Spirit Airlines||Spirit Airlines                       |109533|
|JetBlue Airways                                        |92784 |
|JetBlue Airways||JetBlue Airways                       |68418 |
|Alaska Airlines||Alaska Airlines                       |37066 |
|American Airlines||American Airlines||American Airlines|22761 |
|Delta||Delta||Delta                                    |21274 |
|Spirit Airlines         

In [12]:
fdf_stops_different_airline = df.filter(size(array_distinct(split(col("segmentsAirlineName"), "\|\|"))) > 1)

In [ ]:
fdf_stops_different_airline.groupBy("segmentsAirlineName").agg(count("*").alias("count")).orderBy("count", ascending=False).show(truncate=False)

In [16]:
#Saving the  NonStopFlights Data Into CSV FIle For later use
fdf_nonstop.write.csv("gs://flight-data-bucket-551/processedcsv",header=True, mode="overwrite")


In [13]:
#Saving the  NonStopFlights Data Into Parquet file For later use
fdf_nonstop.write.parquet("gs://flight-data-bucket-551/processed_data", mode="overwrite")


23/12/05 21:37:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [14]:
#Saving the  StopFlights Data Into Parquet file For later use
fdf_stops_different_airline.write.parquet("gs://flight-data-bucket-551/processed_data_fdf_stops_different_airline", mode="overwrite")

In [15]:
#Saving the  NonStopFlights Data Into Parquet file For later use
fdf_stops_same_airline.write.parquet("gs://flight-data-bucket-551/processed_data_fdf_stops_same_airline", mode="overwrite")